In [2]:
import geopandas as gpd

gdf_admin = gpd.read_file("/home/kbj/Downloads/HangJeongDong_ver20241001.geojson")
print(gdf_admin.columns)  # 어떤 행정동 정보가 있는지 확인


Index(['adm_nm', 'adm_cd2', 'sgg', 'sido', 'sidonm', 'sggnm', 'adm_cd',
       'geometry'],
      dtype='object')


In [5]:
import pandas as pd
from shapely.geometry import Point

df_station = pd.read_csv("/home/kbj/Downloads/station_table.csv")

# geometry 컬럼 생성
df_station["geometry"] = df_station.apply(lambda row: Point(row["station_longitude"], row["station_latitude"]), axis=1)
gdf_station = gpd.GeoDataFrame(df_station, geometry="geometry", crs="EPSG:4326")


In [6]:
gdf_admin = gdf_admin.to_crs("EPSG:4326")  # 행정동 경계 좌표계 맞추기
gdf_joined = gpd.sjoin(gdf_station, gdf_admin, how="left", predicate="intersects")


In [11]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# ✅ 1. 대여소 CSV 불러오기
df = pd.read_csv("/home/kbj/Downloads/station_table.csv")

# ✅ 2. GeoDataFrame으로 변환 (geometry 컬럼 만들기)
gdf_stations = gpd.GeoDataFrame(
    df,
    geometry=df.apply(lambda row: Point(row['station_longitude'], row['station_latitude']), axis=1),
    crs="EPSG:4326"
)

# ✅ 3. 행정동 GeoJSON 불러오기
gdf_dong = gpd.read_file("/home/kbj/Downloads/HangJeongDong_ver20241001.geojson")

# ✅ 4. Spatial Join 수행
gdf_joined = gpd.sjoin(gdf_stations, gdf_dong, how="left", predicate="within")

# ✅ 5. 필요한 컬럼만 정리
df_result = gdf_joined[[
    "station_id", "station_gu", "station_dong",
    "station_latitude", "station_longitude", "adm_nm"
]].rename(columns={"adm_nm": "행정동명"})


# ✅ 6. 저장
df_result.to_csv("/home/kbj/Downloads/station_with_admin_dong.csv", index=False)
print("완료! 파일 저장됨: station_with_admin_dong.csv")


완료! 파일 저장됨: station_with_admin_dong.csv


In [10]:
print(gdf_dong.columns)


Index(['adm_nm', 'adm_cd2', 'sgg', 'sido', 'sidonm', 'sggnm', 'adm_cd',
       'geometry'],
      dtype='object')


In [12]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("/home/kbj/Downloads/station_with_admin_dong.csv")

# ✅ 행정동명에서 "서울특별시", "구" 제거
df["행정동_only"] = df["행정동명"].str.replace("서울특별시", "", regex=False)
df["행정동_only"] = df["행정동_only"].str.replace(r"\s*\S+구\s*", "", regex=True)  # 'XX구' 제거

# 결과 확인
print(df[["행정동명", "행정동_only"]].head())


df.to_csv("station_with_dong_only.csv", index=False)


             행정동명 행정동_only
0   서울특별시 마포구 서교동      서교동
1  서울특별시 광진구 자양3동     자양3동
2  서울특별시 양천구 신정7동     신정7동
3  서울특별시 양천구 신월3동     신월3동
4   서울특별시 양천구 목1동      목1동


In [14]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("/home/kbj/Downloads/station_with_dong_only.csv")

# ✅ '행정동_only' 컬럼에서 '·' → '.'로 변경
df["행정동_only"] = df["행정동_only"].str.replace("·", ".", regex=False)

# ✅ 결과 저장
df.to_csv("station_with_dong_only_fixed.csv", index=False)

# ✅ 변경된 상위 5개 확인
print(df[["행정동_only"]].head())


  행정동_only
0      서교동
1     자양3동
2     신정7동
3     신월3동
4      목1동


In [ ]:
import pandas as pd
import mysql.connector

# CSV 불러오기
df = pd.read_csv("/home/kbj/Downloads/station_with_dong_only_fixed.csv")

# MySQL 연결
conn = mysql.connector.connect(
    host="host",
    port=0000,
    user="user",
    password="password",
    database="project1"
)
cursor = conn.cursor()

# 테이블 생성
create_table_sql = """
CREATE TABLE IF NOT EXISTS station_b (
    station_id VARCHAR(20) PRIMARY KEY,
    station_gu VARCHAR(20),
    station_latitude DOUBLE,
    station_longitude DOUBLE,
    station_elevation INT,
    station_dong VARCHAR(50)
);
"""
cursor.execute(create_table_sql)

# 데이터 삽입
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO station_b (station_id, station_gu, station_latitude, station_longitude, station_elevation, station_dong)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['station_id'],
        row['station_gu'],
        row['station_latitude'],
        row['station_longitude'],
        int(row['station_elevation']),
        row['행정동_only']  # 여기 수정됨!
    ))

conn.commit()
cursor.close()
conn.close()
